In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "Guia Spark")

**1- Se tiene un RDD con el registro de notas de los alumnos de la forma (padrón, materia, nota,
fecha). Se pide resolver utilizando PySpark:**
- A. Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.
- B. Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón, promedio).
- C. El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un segundo RDD alumnos con registros (padron, nombre y apellido)

In [2]:
notas = sc.textFile("ej1.csv").map(lambda x: x.split(','))
notas.take(10)

[['10042', 'Quimica', '5', '02/17/2018'],
 ['10074', 'Algo1', '1', '12/15/2017'],
 ['10099', 'Analisis II', '2', '12/02/2018'],
 ['10024', 'Algo2', '7', '08/15/2017'],
 ['10098', 'Taller de programacion', '6', '05/20/2017'],
 ['10015', 'Analisis II', '5', '09/09/2018'],
 ['10037', 'Quimica', '1', '04/21/2019'],
 ['10013', 'Quimica', '10', '05/29/2019'],
 ['10062', 'Analisis II', '9', '04/12/2017'],
 ['10060', 'Algo3', '4', '10/11/2017']]

In [3]:
#A
#  0       1        2      3
#[ padron, materia, nota, fecha ]
aprobadas_ultimos_2_anios = notas.filter(lambda x: (int(x[2])>=4) & (x[3]>"01/01/2017"))
aprobadas_ultimos_2_anios.map(lambda x: x[0]).distinct().count()


101

In [4]:
#B
#armo un RDD de la forma [padron, [nota, 1]]
alumno_nota = notas.map(lambda x: [x[0], [int(x[2]), 1]])
alumno_nota.reduceByKey(lambda x,y: [x[1][0] + y[1][0], x[1][1] + y[1][1]])
alumno_promedio = alumno_nota.map(lambda x: [x[0], x[1][0]/x[1][1]])
alumno_promedio.take(5)

[['10042', 5.0],
 ['10074', 1.0],
 ['10099', 2.0],
 ['10024', 7.0],
 ['10098', 6.0]]

In [5]:

nombres = sc.textFile("ej1_nombres.csv").map(lambda x: x.split(','))
nombres.take(5)

[['10070', 'Esmaria Koppelmann'],
 ['10000', 'Lu Peinke'],
 ['10029', 'Kristyn Timlin'],
 ['10000', 'Phillip Boxer'],
 ['10051', 'Adrea Le Borgne']]

In [6]:
#C
alumno_mejor_promedio = alumno_promedio.reduce(lambda x,y: x if x[1]>y[1] else y)
nombre_alumno_mejor_promedio = nombres.filter(lambda x: x[0] == alumno_mejor_promedio[0])
nombre_alumno_mejor_promedio.collect()

[['10002', 'Kai Jammet'],
 ['10002', 'Arney Lathwell'],
 ['10002', 'Karel Richel'],
 ['10002', 'Eudora De Micoli'],
 ['10002', 'Keefe MacPaik'],
 ['10002', 'Marleen Loxdale'],
 ['10002', 'Prent Silliman'],
 ['10002', 'Kelby Leppard'],
 ['10002', 'Neal Fenne'],
 ['10002', 'Mario Frowen'],
 ['10002', 'Dedra Coston'],
 ['10002', 'Carol Whitman'],
 ['10002', 'Salomo Valeri'],
 ['10002', 'Cchaddie Whitehurst'],
 ['10002', 'Amory Gleadhall'],
 ['10002', 'Lesley Crowhurst'],
 ['10002', 'Dominic Giraudoux'],
 ['10002', 'Amalie Gile'],
 ['10002', 'Rosalinda MacAdie']]

**2- Se tiene un RDD registros de ventas de producto con la forma (fecha de venta, código de
producto, precio de venta) y en otro RDD detalle de los productos con (código de producto,
descripción del producto, categoría). Se pide resolver utilizando PySpark:**
- A. Cuál es el producto más vendido.
- B. Cuál es la categoría de productos más vendida.
- C. Cuál es el top5 de productos más vendidos generando un RDD con (código de producto, descripción, cantidad de ventas)
- D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando para este análisis solo los productos que cuenten con al menos 50 ventas en el último año.
- E. Idem anterior, pero calculando la categoría de productos que registró mayor variación de precios en el último año.

In [3]:
ventas = sc.textFile("ej2.csv").map(lambda x: x.split(','))
productos = sc.textFile("ej2_productos.csv").map(lambda x: x.split(','))
ventas.take(3)

[['11/04/2019', '20', '338'],
 ['12/09/2019', '1', '412'],
 ['07/10/2019', '61', '395']]

In [4]:
productos.take(3)

[['1', 'quis tortor id nulla', 'Computacion'],
 ['2', 'et tempus semper est quam', 'Comida'],
 ['3', 'sed tristique in tempus sit amet sem', 'Computacion']]

In [5]:
#A
cant_vendidos = ventas.map(lambda x: (x[1], 1)).reduceByKey(lambda x,y: x+y)
cant_vendidos.reduce(lambda x,y: x if x[1]>=y[1] else y)

('71', 17)

In [6]:
#B
ventas_ind_prod = ventas.map(lambda x: (x[1],1))
ventas_cat = productos.map(lambda x: (x[0],x[2])).join(ventas_ind_prod)
ventas_x_cat = ventas_cat.map(lambda x: (x[1][0], x[1][1])).reduceByKey(lambda x,y: x+y)
ventas_x_cat.reduce(lambda x,y: x if x[1]>= y[1] else y)[0]

'Utileria'

In [7]:
#C
cant_vendidos_c_descr = productos.map(lambda x: (x[0], x[1])).join(cant_vendidos)
cant_vendidos_c_descr.map(lambda x: [x[0],x[1][0], x[1][1]]).takeOrdered(5, lambda x: -x[2])

[['71', 'nam congue risus semper porta volutpat quam pede', 17],
 ['65', 'id pretium iaculis diam', 16],
 ['17', 'mauris eget massa tempor', 15],
 ['68', 'diam vitae quam suspendisse potenti nullam porttitor lacus at', 15],
 ['77',
  'est lacinia nisi venenatis tristique fusce congue diam id ornare',
  15]]

Este punto debería ser el precio del año anterior menos el precio del año pasado y asi obtener la diferencia no del minimo menos el maximo.

Como takeOrdered devuelve una lista! Tenemos que hacer el Map para ordenarlo de la manera que lo queremos antes.

In [19]:
#D
ventas_ultimo_año = ventas.filter(lambda x: (x[0]>="01/01/2019") & (x[0]<="31/12/2019"))
ventas_mas_50 = ventas_ultimo_año.map(lambda x: (x[1], 1)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1]>= 10) # Lo cambio a 10 por el set de datos
ventas_formateado = ventas.map(lambda x: (x[1], int(x[2]))) #[id, precio]
ventas_mas_50_completo = ventas_formateado.join(ventas_mas_50).map(lambda x: (x[0], x[1][0]))
#[id, precio]
#minimos = ventas_mas_50_completo.reduceByKey(lambda x,y: x if x<=y else y)
minimos = ventas_mas_50_completo.reduceByKey(min)
#maximos = ventas_mas_50_completo.reduceByKey(lambda x,y: x if x>=y else y)
maximos = ventas_mas_50_completo.reduceByKey(max)
minimos_y_maximos = minimos.join(maximos) #[id, (min, max)]
diferencia = minimos_y_maximos.map(lambda x: (x[0], x[1][1]- x[1][0]))
diferencia.reduce(lambda x,y: x if x[1]>y[1] else y)

('56', 442)

In [27]:
#E te la debo


**3- Se tiene un RDD con información de vuelos programados con la forma (número de vuelo,
código de aerolínea, código de aeropuerto de salida, código de aeropuerto de llegada, fecha de
salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada
HH:MM). A su vez, se cuenta con el registro actualizado del estado de los vuelos que fueron
ocurriendo, con la forma (número de vuelo, aerolínea, fecha de salida AAAAMMDD, hora de
salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM, estado). En base al
estado, podría contar con algún dato en blanco, por ejemplo si el vuelo fue cancelado no tendrá
información de fechas y horas, si el vuelo se encuentra aún en curso, no contendrá información
de la llegada. Se pide resolver utilizando PySpark:**
- A. Cuál es el aeropuerto con mayor tránsito.
- B. Cuál es la aerolínea con mayor cantidad de vuelos.
- C. Cuál es la aerolínea con mayor cantidad de cancelaciones.
- D. Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de salida.
- E. Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de llegada.
- F. Cuál es la aerolínea más puntual.
- G. Cuál es el aeropuerto que registra mayor desviación con respecto a los horarios coordinados

In [28]:
#(número de vuelo, código de aerolínea, código de aeropuerto de salida, código de aeropuerto de llegada,
#fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM)

No hay chance que haga un csv para esto. Se hace en papel

**4- Se tiene un RDD con las coordenadas de rectángulos de la forma (x1,x2,y1,y2). Se pide
programar en PySpark un programa que encuentre el rectángulo de superficie mínima que
contiene al punto (w,z)**

In [32]:
rectangulos = sc.textFile("ej4.csv").map(lambda x: [int(n) for n in x.split(",")])
rectangulos.take(5)

[[87, 11, 52, 40],
 [81, 93, 23, 4],
 [58, 52, 85, 11],
 [87, 76, 19, 54],
 [37, 38, 70, 59]]

In [36]:
#[x1, x2, y1, y2]
punto = (25,25) #(w,z) 
#w in [x1,x2] , z in [y1, y2]
def contiene_p(r):
    x1, x2, y1, y2 = r
    contiene_x = (punto[0] > x1) and (punto[0]<x2)
    contiene_y = (punto[1] > y1) and (punto[1]<y2)
    return contiene_x and contiene_y
rectangulos_contienen_punto = rectangulos.filter(contiene_p)
segmentos = rectangulos_contienen_punto.map(lambda x: (abs(x[0]-x[1]), abs(x[2]- x[3])))
areas = segmentos.map(lambda x: x[0]*x[1])
areas.reduce(lambda x,y: x if x<y else y)

189

**5- Se tiene un RDD con libros en donde cada registro es un texto. Se pide obtener todos los anagramas de mas de 7 letras que puedan encontrarse. El formato de salida debe ser una lista de listas en donde cada lista tiene un conjunto de palabras que son anagramas.** Ejemplo:
[[discounter,introduces,reductions],[percussion,supersonic]...]

In [75]:
textos = sc.textFile("alllines.txt")
textos.take(1)

['"ACT I"']

In [95]:
def es_anagrama(palabra, palabra_2):
    if len(palabra) != len(palabra_2) or palabra == palabra_2:
        return False
    for letra in palabra:
        if letra not in palabra_2:
            return False
    for letra in palabra_2:
        if letra not in palabra:
            return False
    return True

def crear_anagramas(lista):
    anagramas = []
    for palabra in lista:
        anagramas_actuales = []
        anagramas_actuales.append(palabra)
        for palabra2 in lista:
            if es_anagrama(palabra, palabra2):
                anagramas_actuales.append(palabra2)
        if len(anagramas_actuales) > 1:
            anagramas.append(anagramas_actuales)
    return anagramas
print(crear_anagramas(["alo", "a", "loa", "al"]))
palabras = textos.flatMap(lambda x: x.split(' ')).map(lambda x: x.strip('"-!?,´.:').strip("'"))
palabras.take(5)

[['alo', 'loa'], ['loa', 'alo']]


['ACT', 'I', 'SCENE', 'I', 'London']

In [96]:
palabras_mas_7_letras = palabras.filter(lambda x: len(x)>7)
palabras_mas_7_letras.take(5)

['LANCASTER', 'WESTMORELAND', 'frighted', 'short-winded', 'commenced']

In [97]:
#palabras_mas_4_letras.map(lambda x: (len(x), [x])).reduceByKey(lambda x,y: x+y)
#palabras_mas_7_letras.distinct().map(lambda x: (len(x), x)).groupByKey().map(lambda x: crear_anagramas(list(x[1]))).take(2)

In [101]:
palabras_mas_7_letras.distinct().map(lambda x: ("".join(sorted(x)), x)).groupByKey()\
    .map(lambda x: list(x[1])).filter(lambda x: len(x)>1).take(5)

[['mischance', 'mechanics'],
 ['brawling', 'warbling'],
 ['unsorted', 'roundest'],
 ['converse', 'conserve'],
 ['dropsies', 'disposer']]